The aim is to read plot EFE

1) Adam, O., T. Bischoff, and T. Schneider, 2016: Seasonal and Interannual Variations of the Energy Flux Equator and ITCZ. Part I: Zonally Averaged ITCZ Position. J. Climate, 29, 3219–3230, https://doi.org/10.1175/JCLI-D-15-0512.1.
2) EFPM : NOT USING NOW. keeping the reference here: Boos, W., Korty, R. Regional energy budget control of the intertropical convergence zone and application to mid-Holocene rainfall. Nature Geosci 9, 892–897 (2016). https://doi.org/10.1038/ngeo2833


In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
from pathlib import Path

# our local module:
import itcz
import myfunctions as mf
import mse_divergence as mdiv 

In [ ]:
# =========================
# User-defined metadata
# =========================

# Base CEDA paths
CEDA_BASE = Path("/badc/cmip6/data/CMIP6")

#Model Names
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    # "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gr",},
    # "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}

#Experiment details
EXPERIMENTS = {
    # "HIST":     {"project": "CMIP",        "scenario": "historical", "color": "black"},
    "SSP245":   {"project": "ScenarioMIP", "scenario": "ssp245"},
    "SSP585":   {"project": "ScenarioMIP", "scenario": "ssp585"},
    "G6solar":  {"project": "GeoMIP",      "scenario": "G6solar"},
    "G6sulfur": {"project": "GeoMIP",      "scenario": "G6sulfur"},
}




In [ ]:
from pathlib import Path

base = CEDA_BASE / "CMIP" / "MOHC" / "UKESM1-0-LL" / "historical" / "r1i1p1f2" / "Amon" / "zg" / "gn" / "latest"
files = sorted(base.glob("*.nc"))
print(len(files), "files found")


In [ ]:
Fdiv_UKESM_2071_2100 = mdiv.compute_Fdiv_for_model_experiments(
    model_name="UKESM1-0-LL",
    model_meta=MODELS["UKESM1-0-LL"],
    experiments=EXPERIMENTS,
    base_dir=CEDA_BASE,
    lon_slice=slice(0, 360),
    season=(1, 12),
    time_slice=slice(2071, 2101),
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

for exp, da in Fdiv_UKESM_2071_2100.items():
    # Compute zonal mean
    zonal_mean = da.mean("lon")
    
    # Plot
    plt.plot(da.lat, zonal_mean, marker='', label=exp)

plt.xlabel("Latitude (°N)")
plt.ylabel(r"W m$^{-1}$")
plt.title("Zonal Mean Annual Mean Divergent MSE Flux (UKESM 2071-2100)")
plt.grid(True)
plt.axhline(0, color='k', linestyle='--', linewidth=0.8)
plt.legend(title="Experiment")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# --- Parameters ---
trop_lat = 60
colors = {
    "SSP245": "blue",
    "SSP585": "red",
    "G6solar": "green",
    "G6sulfur": "orange"
}

# --- Figure and axes ---
fig = plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent([-180, 180, -trop_lat, trop_lat], crs=ccrs.PlateCarree())

# --- Plot zero contours for each experiment ---
for exp, da in Fdiv_UKESM_2071_2100.items():
    # Slice to tropics
    da_tropics = da.sel(lat=slice(-trop_lat, trop_lat))
    
    # Plot zero contour only
    ax.contour(
        da_tropics.lon,
        da_tropics.lat,
        da_tropics,
        levels=[0],
        colors=colors.get(exp, "black"),
        linewidths=2,
        transform=ccrs.PlateCarree(),
        label=exp  # note: contour doesn’t handle label directly
    )

# --- Cartopy features ---
ax.coastlines(linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.3)
ax.add_feature(cfeature.OCEAN, facecolor="white")

# --- Gridlines ---
gl = ax.gridlines(draw_labels=True, linewidth=0.3, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

# --- Manual legend for contours ---
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], color=c, lw=2, label=exp) for exp, c in colors.items()]
ax.legend(handles=legend_elements, title="Experiment", loc="lower left")

ax.set_title("Annual Mean Divergent MSE Flux Zero Contours (2071-2100)")

plt.show()


In [ ]:
#IMMEDIATELY AFTER SAI APPLICATION

Fdiv_UKESM_2020_2050 = mdiv.compute_Fdiv_for_model_experiments(
    model_name="UKESM1-0-LL",
    model_meta=MODELS["UKESM1-0-LL"],
    experiments=EXPERIMENTS,
    base_dir=CEDA_BASE,
    lon_slice=slice(0, 360),
    season=(1, 12),
    time_slice=slice(2020, 2051),
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

for exp, da in Fdiv_UKESM_2020_2050.items():
    # Compute zonal mean
    zonal_mean = da.mean("lon")
    
    # Plot
    plt.plot(da.lat, zonal_mean, marker='', label=exp)

plt.xlabel("Latitude (°N)")
plt.ylabel(r"W m$^{-1}$")
plt.title("Zonal Mean Annual Mean Divergent MSE Flux (UKESM 2020-2050)")
plt.grid(True)
plt.axhline(0, color='k', linestyle='--', linewidth=0.8)
plt.legend(title="Experiment")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# --- Parameters ---
trop_lat = 60
colors = {
    "SSP245": "blue",
    "SSP585": "red",
    "G6solar": "green",
    "G6sulfur": "orange"
}

# --- Figure and axes ---
fig = plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent([-180, 180, -trop_lat, trop_lat], crs=ccrs.PlateCarree())

# --- Plot zero contours for each experiment ---
for exp, da in Fdiv_UKESM_2020_2050.items():
    # Slice to tropics
    da_tropics = da.sel(lat=slice(-trop_lat, trop_lat))
    
    # Plot zero contour only
    ax.contour(
        da_tropics.lon,
        da_tropics.lat,
        da_tropics,
        levels=[0],
        colors=colors.get(exp, "black"),
        linewidths=2,
        transform=ccrs.PlateCarree(),
        label=exp  # note: contour doesn’t handle label directly
    )

# --- Cartopy features ---
ax.coastlines(linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.3)
ax.add_feature(cfeature.OCEAN, facecolor="white")

# --- Gridlines ---
gl = ax.gridlines(draw_labels=True, linewidth=0.3, color="gray", alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

# --- Manual legend for contours ---
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], color=c, lw=2, label=exp) for exp, c in colors.items()]
ax.legend(handles=legend_elements, title="Experiment", loc="lower left")

ax.set_title("Annual Mean Divergent MSE Flux Zero Contours (2020-2050)")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.lines import Line2D

# --- Parameters ---
trop_lat = 60

# --- Extract G6sulfur ---
Fdiv_2020 = Fdiv_UKESM_2020_2050["G6sulfur"]
Fdiv_2071 = Fdiv_UKESM_2071_2100["G6sulfur"]

# --- Slice to tropics ---
Fdiv_2020_trop = Fdiv_2020.sel(lat=slice(-trop_lat, trop_lat))
Fdiv_2071_trop = Fdiv_2071.sel(lat=slice(-trop_lat, trop_lat))

# --- Figure and axes ---
fig = plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent([-180, 180, -trop_lat, trop_lat], crs=ccrs.PlateCarree())

# --- Zero contours ---
ax.contour(
    Fdiv_2020_trop.lon,
    Fdiv_2020_trop.lat,
    Fdiv_2020_trop,
    levels=[0],
    colors="orange",
    linewidths=2,
    linestyles="solid",
    transform=ccrs.PlateCarree(),
)

ax.contour(
    Fdiv_2071_trop.lon,
    Fdiv_2071_trop.lat,
    Fdiv_2071_trop,
    levels=[0],
    colors="brown",
    linewidths=2,
    linestyles="dashed",
    transform=ccrs.PlateCarree(),
)

# --- Cartopy features ---
ax.coastlines(linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.3)
ax.add_feature(cfeature.OCEAN, facecolor="white")

# --- Gridlines ---
gl = ax.gridlines(draw_labels=True, linewidth=0.3, color="gray",
                  alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

# --- Legend ---
legend_elements = [
    Line2D([0], [0], color="orange", lw=2, linestyle="solid",
           label="G6sulfur 2020–2050"),
    Line2D([0], [0], color="brown", lw=2, linestyle="dashed",
           label="G6sulfur 2071–2100"),
]
ax.legend(handles=legend_elements, loc="lower left")

ax.set_title("Zero Contours of Divergent MSE Flux (G6sulfur)")

plt.show()


In [ ]:
import itcz
ITCZ_Adam_UKESM_2071_2100 = itcz.compute_adam_ITCZ(
    model_name="UKESM1-0-LL",
    model_meta=MODELS["UKESM1-0-LL"],
    experiments=EXPERIMENTS,
    base_dir=CEDA_BASE,
    lon_slice=slice(0, 360),
    season=(1, 12),
    time_slice=slice(2071, 2101),
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.lines import Line2D

# --- Parameters ---
trop_lat = 60

# --- Extract data ---
Fdiv = Fdiv_UKESM_2071_2100["G6sulfur"]
ITCZ = ITCZ_Adam_UKESM_2071_2100["G6sulfur"]

# --- Slice Fdiv to tropics ---
Fdiv_trop = Fdiv.sel(lat=slice(-trop_lat, trop_lat))

# --- ITCZ mean over years ---
itcz_mean = ITCZ.mean("year")

# --- Wrap longitudes (0–360 → -180–180) ---
lon = itcz_mean.lon.values
lon_wrap = ((lon + 180) % 360) - 180
sort_idx = np.argsort(lon_wrap)

lon_sorted  = lon_wrap[sort_idx]
itcz_sorted = itcz_mean.values[sort_idx]

# --- Figure and axes ---
fig = plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent([-180, 180, -trop_lat, trop_lat], crs=ccrs.PlateCarree())

# --- Fdiv zero contour (G6sulfur, 2071–2100) ---
ax.contour(
    Fdiv_trop.lon,
    Fdiv_trop.lat,
    Fdiv_trop,
    levels=[0],
    colors="brown",
    linewidths=2,
    transform=ccrs.PlateCarree(),
)

# --- Adam ITCZ line ---
ax.plot(
    lon_sorted,
    itcz_sorted,
    color="black",
    linewidth=2.5,
    transform=ccrs.PlateCarree(),
    label="Adam ITCZ (mean, 2071–2100)",
)

# --- Reference equator ---
ax.axhline(0, color="k", linewidth=0.6, linestyle="--")

# --- Cartopy features ---
ax.coastlines(linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.3)
ax.add_feature(cfeature.OCEAN, facecolor="white")

# --- Gridlines ---
gl = ax.gridlines(draw_labels=True, linewidth=0.3, color="gray",
                  alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False

# --- Legend ---
legend_elements = [
    Line2D([0], [0], color="brown", lw=2, label="EFE"),
    Line2D([0], [0], color="black", lw=2.5, label="Adam ITCZ"),
]
ax.legend(handles=legend_elements, loc="lower left")

ax.set_title("G6sulfur (2071–2100): Wei_Bordoni_2020_EFE and Adam_2016_ITCZ")

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.lines import Line2D

# =====================
# Parameters
# =====================
trop_lat   = 60
num_levels = 21

# =====================
# Extract data
# =====================
Fdiv = Fdiv_UKESM_2071_2100["G6sulfur"]
ITCZ = ITCZ_Adam_UKESM_2071_2100["G6sulfur"]

# =====================
# Slice to tropics
# =====================
Fdiv_trop = Fdiv.sel(lat=slice(-trop_lat, trop_lat))

# =====================
# Adam ITCZ mean (over years)
# =====================
itcz_mean = ITCZ.mean("year")

# --- Wrap longitudes (0–360 → -180–180) ---
lon = itcz_mean.lon.values
lon_wrap = ((lon + 180) % 360) - 180
sort_idx = np.argsort(lon_wrap)

lon_sorted  = lon_wrap[sort_idx]
itcz_sorted = itcz_mean.values[sort_idx]

# =====================
# Figure and axes
# =====================
fig = plt.figure(figsize=(14, 6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent([-180, 180, -trop_lat, trop_lat], crs=ccrs.PlateCarree())

# =====================
# Divergent MSE shading
# =====================
vmax = np.nanmax(np.abs(Fdiv_trop))
levels = np.linspace(-vmax, vmax, num_levels)

cf = ax.contourf(
    Fdiv_trop.lon,
    Fdiv_trop.lat,
    Fdiv_trop,
    levels=levels,
    cmap="RdBu_r",
    extend="both",
    transform=ccrs.PlateCarree(),
)

# =====================
# Zero contour (EFE)
# =====================
ax.contour(
    Fdiv_trop.lon,
    Fdiv_trop.lat,
    Fdiv_trop,
    levels=[0],
    colors="black",
    linewidths=2.5,
    transform=ccrs.PlateCarree(),
)

# =====================
# Adam ITCZ (thick green)
# =====================
ax.plot(
    lon_sorted,
    itcz_sorted,
    color="green",
    linewidth=3.5,
    transform=ccrs.PlateCarree(),
)

# =====================
# Reference equator
# =====================
ax.axhline(0, color="k", linewidth=0.6, linestyle="--")

# =====================
# Cartopy features
# =====================
ax.coastlines(linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.3)
ax.add_feature(cfeature.OCEAN, facecolor="white")

# =====================
# Gridlines
# =====================
gl = ax.gridlines(
    draw_labels=True,
    linewidth=0.3,
    color="gray",
    alpha=0.5,
    linestyle="--",
)
gl.top_labels = False
gl.right_labels = False

# =====================
# Legend
# =====================
legend_elements = [
    Line2D([0], [0], color="black", lw=2.5, label="EFE (Fdiv = 0)"),
    Line2D([0], [0], color="green", lw=3.5, label="Adam ITCZ"),
]
ax.legend(handles=legend_elements, loc="lower left")

# =====================
# Colorbar
# =====================
cbar = plt.colorbar(
    cf,
    ax=ax,
    orientation="horizontal",
    pad=0.06,
    aspect=40,
)
cbar.set_label("Divergent MSE Flux (W m$^{-2}$)")

# =====================
# Title
# =====================
ax.set_title(
    "G6sulfur (2071–2100): Divergent MSE Flux, Energetic Equator, and Adam ITCZ"
)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.lines import Line2D

# =====================
# Parameters
# =====================
trop_lat   = 90
num_levels = 21

# =====================
# Extract data
# =====================
Fdiv = Fdiv_UKESM_2071_2100["SSP245"]
ITCZ = ITCZ_Adam_UKESM_2071_2100["SSP245"]

# =====================
# Slice to tropics
# =====================
Fdiv_trop = Fdiv.sel(lat=slice(-trop_lat, trop_lat))

# =====================
# Adam ITCZ mean (over years)
# =====================
itcz_mean = ITCZ.mean("year")

# --- Wrap longitudes (0–360 → -180–180) ---
lon = itcz_mean.lon.values
lon_wrap = ((lon + 180) % 360) - 180
sort_idx = np.argsort(lon_wrap)

lon_sorted  = lon_wrap[sort_idx]
itcz_sorted = itcz_mean.values[sort_idx]

# =====================
# Figure and axes
# =====================
fig = plt.figure(figsize=(14, 6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
ax.set_extent([-180, 180, -trop_lat, trop_lat], crs=ccrs.PlateCarree())

# =====================
# Divergent MSE shading
# =====================
vmax = np.nanmax(np.abs(Fdiv_trop))
levels = np.linspace(-vmax, vmax, num_levels)

cf = ax.contourf(
    Fdiv_trop.lon,
    Fdiv_trop.lat,
    Fdiv_trop,
    levels=levels,
    cmap="RdBu_r",
    extend="both",
    transform=ccrs.PlateCarree(),
)

# =====================
# Zero contour (EFE)
# =====================
ax.contour(
    Fdiv_trop.lon,
    Fdiv_trop.lat,
    Fdiv_trop,
    levels=[0],
    colors="black",
    linewidths=2.5,
    transform=ccrs.PlateCarree(),
)

# =====================
# Adam ITCZ (thick green)
# =====================
ax.plot(
    lon_sorted,
    itcz_sorted,
    color="green",
    linewidth=3.5,
    transform=ccrs.PlateCarree(),
)

# =====================
# Reference equator
# =====================
ax.axhline(0, color="k", linewidth=0.6, linestyle="--")

# =====================
# Cartopy features
# =====================
ax.coastlines(linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.3)
ax.add_feature(cfeature.OCEAN, facecolor="white")

# =====================
# Gridlines
# =====================
gl = ax.gridlines(
    draw_labels=True,
    linewidth=0.3,
    color="gray",
    alpha=0.5,
    linestyle="--",
)
gl.top_labels = False
gl.right_labels = False

# =====================
# Legend
# =====================
legend_elements = [
    Line2D([0], [0], color="black", lw=2.5, label="EFE (Fdiv = 0)"),
    Line2D([0], [0], color="green", lw=3.5, label="Adam ITCZ"),
]
ax.legend(handles=legend_elements, loc="lower left")

# =====================
# Colorbar
# =====================
cbar = plt.colorbar(
    cf,
    ax=ax,
    orientation="horizontal",
    pad=0.06,
    aspect=40,
)
cbar.set_label("Divergent MSE Flux (W m$^{-2}$)")

# =====================
# Title
# =====================
ax.set_title(
    "SSP245 (2071–2100): Divergent MSE Flux, Energetic Equator, and Adam ITCZ"
)

plt.tight_layout()
plt.show()


In [ ]:
plt.plot(Fdiv_UKESM_2071_2100["SSP245"].lat, Fdiv_UKESM_2071_2100["SSP245"], marker='', label=exp)